<a href="https://colab.research.google.com/github/quinbez/BERT-For-Multi-Class-Text-Classifier/blob/main/BERT_for_Multi_class_text_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BERT FOR TEXT CLASSIFICATION**

# **Importing the libraries**

In [ ]:
# Importing Libraries
import re
import torch
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch.nn as nn
from transformers import BertModel
from transformers import BertTokenizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
# Learning rate for training
lr = 1e-3

# Sequence length (often used for sequences in NLP)
seq_len = 20

# Dropout rate for regularization
dropout = 0.5

# Number of training epochs
num_epochs = 2

# Column name for labels in the dataset
label_col = "Product"

# Path to the file containing tokenized data
tokens_path = "Output/tokens.pkl"

# Path to the file containing label data
labels_path = "Output/labels.pkl"

# Path to the input data file (complaints dataset)
data_path = "Input/complaints.csv"

# Path to save the pre-trained model
model_path = "Output/bert_pre_trained.pth"

# Column name for text data in the dataset
text_col_name = "Consumer complaint narrative"

# Path to the label encoder file
label_encoder_path = "Output/label_encoder.pkl"

# Mapping of product categories to their corresponding labels
product_map = {
    'Vehicle loan or lease': 'vehicle_loan',
    'Credit reporting, credit repair services, or other personal consumer reports': 'credit_report',
    'Credit card or prepaid card': 'card',
    'Money transfer, virtual currency, or money service': 'money_transfer',
    'virtual currency': 'money_transfer',
    'Mortgage': 'mortgage',
    'Payday loan, title loan, or personal loan': 'loan',
    'Debt collection': 'debt_collection',
    'Checking or savings account': 'savings_account',
    'Credit card': 'card',
    'Bank account or service': 'savings_account',
    'Credit reporting': 'credit_report',
    'Prepaid card': 'card',
    'Payday loan': 'loan',
    'Other financial service': 'others',
    'Virtual currency': 'money_transfer',
    'Student loan': 'loan',
    'Consumer Loan': 'loan',
    'Money transfers': 'money_transfer'
}

# **Save and Load the File**

In [ ]:
import pickle

def save_file(name, obj):
    """
    Function to save an object as a pickle file.

    :param name: The name of the pickle file to save.
    :param obj: The object to be saved.
    """
    with open(name, 'wb') as f:
        pickle.dump(obj, f)

def load_file(name):
    """
    Function to load a pickle object.

    :param name: The name of the pickle file to load.
    :return: The loaded object from the pickle file.
    """
    return pickle.load(open(name, "rb"))

# **Preprocessing the data**

In [ ]:
data = pd.read_csv(data_path)

In [ ]:
data.dropna(subset=[text_col_name], inplace=True)

In [ ]:

data.replace({label_col: product_map}, inplace=True)

# **Encode labels**

In [ ]:
# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the LabelEncoder to the labels in the 'label_col' column of your data
label_encoder.fit(data[label_col])

# Transform (encode) the labels in the 'label_col' column into numerical values
labels = label_encoder.transform(data[label_col])

In [ ]:

save_file(labels_path, labels)
save_file(label_encoder_path, label_encoder)

# **Process the text column**

In [ ]:
# Extract text data from 'data' into a list
text_list = list(data[text_col_name])

In [ ]:
len(input_text)

# **Convert text to lower case**

In [ ]:
# Convert text to lowercase with progress bar
input_text = [i.lower() for i in tqdm(input_text)]

# **Remove punctuations except apostrophe**

In [ ]:
# Replace non-alphanumeric characters with spaces in text with progress bar
input_text = [re.sub(r"[^\w\d'\s]+", " ", i) for i in tqdm(input_text)]

# **Remove digits**

In [ ]:
# Remove digits from text with progress bar
input_text = [re.sub("\d+", "", i) for i in tqdm(input_text)]

# **Remove more than one consecutive instance of 'x'**

In [ ]:
# Remove consecutive 'x' characters (2 or more) from text with progress bar
input_text = [re.sub(r'[x]{2,}', "", i) for i in tqdm(input_text)]

# **Remove multiple spaces with single space**

In [ ]:
# Replace multiple spaces with a single space in text with progress bar
input_text = [re.sub(' +', ' ', i) for i in tqdm(input_text)]

# **Tokenize the text**

In [ ]:
# Initialize a tokenizer using the 'bert-base-cased' model
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [ ]:
input_text[0]

In [ ]:
# Tokenize the first text sample using the tokenizer
sample_tokens = tokenizer(input_text[0], padding="max_length", max_length=seq_len, truncation=True, return_tensors="pt")

In [ ]:
sample_tokens

In [ ]:
sample_tokens["input_ids"]

In [ ]:
sample_tokens["attention_mask"]

In [ ]:
# Tokenize each text in the input_text list with specified settings and create a list of tokens

tokens = [tokenizer(i, padding="max_length", max_length=seq_len, truncation=True, return_tensors="pt") for i in tqdm(input_text)]


# **Save the tokens**

In [ ]:
save_file(tokens_path, tokens)

# **Custom Dataset**

In [ ]:
import torch

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, tokens, labels):
        """
        Initialize a TextDataset.

        Args:
            tokens (list): List of tokenized text data.
            labels (list): List of corresponding labels.

        This class is designed to work as a PyTorch Dataset, which means it can be used with PyTorch's DataLoader for efficient data loading during training and evaluation.
        """
        self.tokens = tokens  # List of tokenized text data
        self.labels = labels  # List of corresponding labels

    def __len__(self):
        """
        Get the total number of data points in the dataset.

        Returns:
            int: Number of data points in the dataset.
        """
        return len(self.tokens)

    def __getitem__(self, idx):
        """
        Get a specific data point (a pair of text data and its label) from the dataset.

        Args:
            idx (int): Index of the data point to retrieve.

        Returns:
            tuple: A tuple containing the label and tokenized text data for the specified data point.
        """
        return self.labels[idx], self.tokens[idx]

# **BERT CLASSIFIER MODEL**

In [ ]:
import torch
import numpy as np
import torch.nn as nn
from tqdm import tqdm
from transformers import BertModel
from sklearn.metrics import accuracy_score

class BertClassifier(nn.Module):

    def __init__(self, dropout, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        for param in self.bert.parameters():
            param.requires_grad = False  # Freeze BERT parameters
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, num_classes)  # 768 is the BERT hidden size
        self.activation = nn.ReLU()

    def forward(self, input_ids, attention_mask):
        # Perform a forward pass through the model
        _, bert_output = self.bert(input_ids=input_ids,
                                   attention_mask=attention_mask,
                                   return_dict=False)
        dropout_output = self.activation(self.dropout(bert_output))
        final_output = self.linear(dropout_output)
        return final_output

# **Training The Model**

In [ ]:
def train(train_loader, valid_loader, model, criterion, optimizer,
          device, num_epochs, model_path):
    """
    Function to train the model
    :param train_loader: Data loader for the train dataset
    :param valid_loader: Data loader for the validation dataset
    :param model: Model object
    :param criterion: Loss function
    :param optimizer: Optimizer
    :param device: CUDA or CPU
    :param num_epochs: Number of epochs
    :param model_path: Path to save the model
    """
    best_loss = 1e8
    for i in range(num_epochs):
        print(f"Epoch {i+1} of {num_epochs}")
        valid_loss, train_loss = [], []
        model.train()  # Set the model in training mode
        # Train loop
        for batch_labels, batch_data in tqdm(train_loader):
            input_ids = batch_data["input_ids"]
            attention_mask = batch_data["attention_mask"]
            # Move data to GPU if available
            batch_labels = batch_labels.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            input_ids = torch.squeeze(input_ids, 1)
            # Forward pass
            batch_output = model(input_ids, attention_mask)
            batch_output = torch.squeeze(batch_output)
            # Calculate loss
            loss = criterion(batch_output, batch_labels)
            train_loss.append(loss.item())
            optimizer.zero_grad()
            # Backward pass
            loss.backward()
            # Gradient update step
            optimizer.step()
        model.eval()  # Set the model in evaluation mode
        # Validation loop
        for batch_labels, batch_data in tqdm(valid_loader):
            input_ids = batch_data["input_ids"]
            attention_mask = batch_data["attention_mask"]
            # Move data to GPU if available
            batch_labels = batch_labels.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            input_ids = torch.squeeze(input_ids, 1)
            # Forward pass
            batch_output = model(input_ids, attention_mask)
            batch_output = torch.squeeze(batch_output)
            # Calculate loss
            loss = criterion(batch_output, batch_labels)
            valid_loss.append(loss.item())
        t_loss = np.mean(train_loss)
        v_loss = np.mean(valid_loss)
        print(f"Train Loss: {t_loss}, Validation Loss: {v_loss}")
        if v_loss < best_loss:
            best_loss = v_loss
            # Save the model if validation loss improves
            torch.save(model.state_dict(), model_path)
        print(f"Best Validation Loss: {best_loss}")

# **Testing the Model**

In [ ]:
def test(test_loader, model, criterion, device):
    """
    Function to test the model
    :param test_loader: Data loader for the test dataset
    :param model: Model object
    :param criterion: Loss function
    :param device: CUDA or CPU
    """
    model.eval()  # Set the model in evaluation mode
    test_loss = []
    test_accu = []
    for batch_labels, batch_data in tqdm(test_loader):
        input_ids = batch_data["input_ids"]
        attention_mask = batch_data["attention_mask"]
        # Move data to GPU if available
        batch_labels = batch_labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        input_ids = torch.squeeze(input_ids, 1)
        # Forward pass
        batch_output = model(input_ids, attention_mask)
        batch_output = torch.squeeze(batch_output)
        # Calculate loss
        loss = criterion(batch_output, batch_labels)
        test_loss.append(loss.item())
        batch_preds = torch.argmax(batch_output, axis=1)
        # Move predictions to CPU
        if torch.cuda.is_available():
            batch_labels = batch_labels.cpu()
            batch_preds = batch_preds.cpu()
        # Compute accuracy
        test_accu.append(accuracy_score(batch_labels.detach().numpy(),
                                        batch_preds.detach().numpy()))
    test_loss = np.mean(test_loss)
    test_accu = np.mean(test_accu)
    print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc})

# **Save and Load the File**

# **Load the files**

In [ ]:
# Load token data, labels, and label encoder
tokens = load_file(tokens_path)
labels = load_file(labels_path)
label_encoder = load_file(label_encoder_path)

# Get the number of classes from the label encoder
num_classes = len(label_encoder.classes_)

# **Split data into train, validation and test sets**

In [ ]:
# Split data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(tokens, labels, test_size=0.2)

# Further split the training data into training and validation sets (60% train, 20% validation)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25)

# **Create PyTorch datasets**

In [ ]:
# Create training, validation, and test datasets
train_dataset = TextDataset(X_train, y_train)
valid_dataset = TextDataset(X_valid, y_valid)
test_dataset = TextDataset(X_test, y_test)

# **Create data loaders**

In [ ]:
# Create data loaders for training, validation, and test datasets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16)

# **Create model object**

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available()
                     else "cpu")

In [ ]:
# Create a BERT-based classifier model
model = BertClassifier(dropout, num_classes)

# **Define loss function and optimizer**

In [ ]:
Define CrossEntropyLoss for the criterion
criterion = torch.nn.CrossEntropyLoss()

# Initialize an Adam optimizer for model parameters with a specified learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# **Move the model to GPU if available**

In [ ]:
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

# **Training loop**


In [ ]:
# Train the model
train(train_loader, valid_loader, model, criterion, optimizer, device, num_epochs, model_path)

# **Test the model**

In [ ]:
test(test_loader, model, criterion, device)

# **Predict on new text**

In [ ]:

input_text = '''I am a victim of Identity Theft & currently have an Experian account that
I can view my Experian Credit Report and getting notified when there is activity on
my Experian Credit Report. For the past 3 days I've spent a total of approximately 9
hours on the phone with Experian. Every time I call I get transferred repeatedly and
then my last transfer and automated message states to press 1 and leave a message and
someone would call me. Every time I press 1 I get an automatic message stating than you
before I even leave a message and get disconnected. I call Experian again, explain what
is happening and the process begins again with the same end result. I was trying to have
this issue attended and resolved informally but I give up after 9 hours. There are hard
hit inquiries on my Experian Credit Report that are fraud, I didn't authorize, or recall
and I respectfully request that Experian remove the hard hit inquiries immediately just
like they've done in the past when I was able to speak to a live Experian representative
in the United States. The following are the hard hit inquiries : BK OF XXXX XX/XX/XXXX
XXXX XXXX XXXX  XX/XX/XXXX XXXX  XXXX XXXX  XX/XX/XXXX XXXX  XX/XX/XXXX XXXX  XXXX
XX/XX/XXXX'''

In [ ]:
# Convert text to lowercase
input_text = input_text.lower()

# Replace non-alphanumeric characters with spaces
input_text = re.sub(r"[^\w\d'\s]+", " ", input_text)

# Remove digits from text
input_text = re.sub("\d+", "", input_text)

# Remove consecutive 'x' characters (2 or more)
input_text = re.sub(r'[x]{2,}', "", input_text)

# Replace multiple spaces with a single space
input_text = re.sub(' +', ' ', input_text)

In [ ]:
# Initialize a tokenizer using the 'bert-base-cased' model
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [ ]:
# Tokenize the input_text with specified settings
tokens = tokenizer(input_text, padding="max_length", max_length=seq_len, truncation=True, return_tensors="pt")

In [ ]:
# Get input IDs from the tokenized data
input_ids = tokens["input_ids"]

# Get attention mask from the tokenized data
attention_mask = tokens["attention_mask"]

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available()
                     else "cpu")

In [ ]:
# Move input IDs to the specified device (GPU or CPU)
input_ids = input_ids.to(device)

# Move attention mask to the specified device (GPU or CPU)
attention_mask = attention_mask.to(device)

In [ ]:
# Squeeze the input_ids tensor along the specified dimension (1)
input_ids = torch.squeeze(input_ids, 1)

In [ ]:
# Load the label encoder from a file
label_encoder = load_file(label_encoder_path)

# Get the number of classes from the label encoder
num_classes = len(label_encoder.classes_)

In [ ]:
# Create a model object
model = BertClassifier(dropout, num_classes)

# Load trained weights into the model
model.load_state_dict(torch.load(model_path))

# Move the model to GPU if available
if torch.cuda.is_available():
    model = model.cuda()

# Forward pass and get model output
out = torch.squeeze(model(input_ids, attention_mask))

# Find the predicted class using the label encoder
prediction = label_encoder.classes_[torch.argmax(out)]
print(f"Predicted Class: {prediction}")